Hướng dẫn sử dụng:

Tải Data dạng XML lên GG Colab

Chạy lần lượt các phần.

Nhập tên Data ở phần 5 thay thế cho /any.xml

Phần 1)   Đo độ tương đồng giữa các định nghĩa tiếng anh

In [7]:
# 1) từ 4h40pm ngày 03-09-2024 đưa đo độ tương đồng của Quang vào cùng file với từ điển WORDNET  ----------------------------OK
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch

# Hàm để lấy embeddings từ mô hình GTE-Large
def get_embeddings(texts):
    tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-large")
    model = AutoModel.from_pretrained("thenlper/gte-large")

    # Tokenize the input texts
    batch_dict = tokenizer(texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**batch_dict)

    # Average pooling
    last_hidden_states = outputs.last_hidden_state
    attention_mask = batch_dict['attention_mask']
    embeddings = average_pool(last_hidden_states, attention_mask)

    # Normalize embeddings
    embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

    return embeddings.numpy()

# Hàm để thực hiện average pooling trên các embeddings
def average_pool(last_hidden_states: torch.Tensor,
                 attention_mask: torch.Tensor) -> torch.Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Hàm để tìm cặp mô tả có độ tương đồng cao
def find_high_similarity_pairs(list1, list2, threshold=0.83):

    # Trích xuất các đn tiếng anh trong danh sách `list1` và `list2`
    descriptions1 = [item['em'] for item in list1] ;
    descriptions2 = [item['em'] for item in list2] ;

    # Lấy embeddings cho các mô tả trong cả hai danh sách
    embeddings1 = get_embeddings(descriptions1) ;
    embeddings2 = get_embeddings(descriptions2) ;

    # Tính toán ma trận độ tương đồng giữa các embeddings
    similarities = cosine_similarity(embeddings1, embeddings2)

    # In ma trận độ tương đồng với các giá trị > threshold
    filtered_similarities = np.where(similarities > threshold, similarities, 0)

    # Khởi tạo danh sách lưu trữ các cặp tương đồng cao
    pairs = []

    # Tạo hai tập hợp để theo dõi các chỉ số đã được ghép cặp
    matched1 = set()
    matched2 = set()

    # Duyệt qua từng hàng trong ma trận độ tương đồng
    for i, row in enumerate(similarities):
        # Duyệt qua từng cột trong hàng hiện tại
        for j, similarity in enumerate(row):
            # Kiểm tra nếu độ tương đồng lớn hơn ngưỡng và cả hai mô tả chưa được ghép cặp
            if similarity > threshold and i not in matched1 and j not in matched2:
                # Thêm các chỉ số vào các tập hợp đã được ghép cặp
                matched1.add(i)
                matched2.add(j)
                # Thêm cặp vào danh sách các cặp tương đồng cao
                pairs.append({
                    'Index_List1': i,
                    'Index_List2': j,
                    'Similarity': similarity
                })
                # In ra các cặp có độ tương đồng cao nhất
                print(f"Added pair: Index_List1={i}, Index_List2={j}, Similarity={similarity}")

    return pairs    # Trả về danh sách các cặp mô tả có độ tương đồng cao


Phần 2)  Từ điển EEV Oxford, file XML

In [8]:
import xml.etree.ElementTree as et
from lxml import etree
import re
import json


def wordlist(file_paths):
  word_list=[]
  for file_path in file_paths:
    print("File_path = ", file_path)
    tree=et.parse(file_path)
    root = tree.getroot()
    for i in root.iter():
      if i.tag == "runhd":
        word_list.append(i.text.strip())
  return root, word_list


def findword(root, nhap_1_tu):
  kq = root.findall(".//runhd") ;
  for tu in kq:
    if tu.text == nhap_1_tu:
      break
  return tu

# hàm tìm cha của 1 node
def findfather(root, ptu): ##### thêm đối số root ngày 22-09-24
  papa=None
  for i in root.iter():
    if ptu in i: papa=i; break
  return papa

def textprocess (chuoi):
  newchuoi=chuoi.strip()
  kytudb = r'[~@#$%^&*<>?:{}|\\\[\]]'
  newchuoi= re.sub(r'/[^/]+/'," ", newchuoi)
  newchuoi= re.sub(kytudb," ",newchuoi)
  newchuoi= re.sub(r'\s{2,}', " ", newchuoi)
  newchuoi=newchuoi.replace(" ," , ",")
  newchuoi=newchuoi.replace(" , " , ", ")
  newchuoi=newchuoi.replace(" .",".")
  newchuoi=newchuoi.replace(" . ",". ")
  return newchuoi


# Tìm các chuỗi nằm dưới 1 node
def alltext_under1node(element):
    text_list = []

    for elem in element.iter():
        if elem.text is not None:
            text_list.append(elem.text.strip())
        if elem.tail is not None:
            text_list.append(elem.tail.strip())

    result = ' '.join(text_list)
    return result

# hàm lọc ra các text định nghĩa tiếng anh
def text_outside_children(nodecha):
  full_text = ''.join(nodecha.itertext())

  symbol=nodecha.find(".//symbol")
  if symbol is not None:
    symbol_texts=''.join(symbol.itertext())  #
    full_text=full_text.replace(symbol_texts,"").replace("","")

  txt_v_s_srf=nodecha.find(".//txt_v_s_srf");
  if txt_v_s_srf is not None:
    txt_v_s_srf_texts=''.join(txt_v_s_srf.itertext()) ;
    full_text=full_text.replace(txt_v_s_srf_texts,"")

  return ' '.join(full_text.split()).strip()

def meaningex(root,d_ud): # Thêm đối số root vào 22-09-24 # hàm nhập vào 1 mảng 1 node <d> hay <ud>, trả về nghĩa anh, việt, các ví dụ
 #print(" + Nghĩa tiếng anh : ", end="")
  em = ""
  dhs=""
  if d_ud.tag in ["d", "ud"]:
    em = text_outside_children(d_ud) ;
    if "" in em: em = em[0:em.find("")]
  elif d_ud.tag in ["xr"]:
    for dud in d_ud.iter():
      text= dud.text.strip() if dud.text else ""
      tail= dud.tail.strip() if dud.tail else ""
      #print(text, tail, " ", end="")
      em = em + text + tail
    em=em.strip();
  #print(" + nghĩa tiếng anh trong hàm meaningex = ",em) # in nghĩa tiếng anh

  # nghĩa tiếng Việt
  vm="";
  #print("") ;
  txt_v_s_srf = d_ud.find(".//txt_v_s_srf");
  list_z = findfather(root,d_ud).findall(".//z");
  if txt_v_s_srf is not None:
    #print(" + Nghĩa tiếng Việt: ",end="")
    for txt in txt_v_s_srf.iter():
      text= txt.text.strip() if txt.text else ""
      tail= txt.tail.strip() if txt.tail else ""
      vm = vm + text + tail ;

  elif "" in d_ud.text:
    stt = d_ud.text.find("") ;
    vm = vm + d_ud.text[stt:]

  #print ("vm= ", vm)
  #print()
  vidu=findfather(root,d_ud).findall(".//x");
  ex=[]
  for x in vidu:
    kq1=''.join(x.itertext())
    kq2=' '.join(kq1.split())
    ex.append(kq2 ) ;
  #print(" + Ví dụ ex = ", ex)

  return em, vm, ex


def dictionary_1word_xml(root,word): # hàm nhận đối số là root của file xml và 1 từ, trả về thông tin là 1 dictionary có các key là phiên âm và các từ loại
  thongtin = {}
  runhd=findword(root, word)
  father = findfather(root,runhd);
  grand = findfather(root,father);
  greatgrand = findfather(root,grand);

  pronunciation=[]
  phienam=father.findall(".//i");
  for j in phienam:
    #print("phiên âm = /", j.text,"/")
    pronunciation.append(j.text)

  thongtin["pronunciation"]=pronunciation
  p_g=greatgrand.findall(".//p-g") ;
  if not p_g :
    if greatgrand.find(".//z_p")is not None:
      wordtype = greatgrand.find(".//z_p").text.strip()
    else: return thongtin
    thongtin[wordtype] =  []
    n_g = greatgrand.findall(".//n-g") ;
    if not n_g :
      if greatgrand.find(".//d") : d_ud = greatgrand.find(".//d") ;
      elif greatgrand.find(".//ud") : d_ud = greatgrand.find(".//ud");
      elif greatgrand.find(".//xr") : d_ud = greatgrand.find(".//xr") ;
      elif greatgrand.find(".//h-g") : d_ud = greatgrand.find(".//h-g") ;
      if d_ud :
        em, vm, ex = meaningex(root, d_ud)
        thongtin[wordtype].append({"em":em, "vm":vm, "ex":ex})
    else:
      for ng  in n_g:
        if findfather(root, ng).tag not in ['pv-g','id-g']:
          zn=ng.find(".//zn")
          d_ud= ng.find(".//d");
          if ng.find(".//d") :    d_ud = ng.find(".//d");
          elif ng.find(".//ud") : d_ud = ng.find(".//ud");
          elif ng.find(".//xr") : d_ud = ng.find(".//xr");
          if d_ud:
            em, vm, ex = meaningex( root, d_ud)  ;
            thongtin[wordtype].append({"em":em, "vm":vm, "ex":ex})
        d_ud=None
  else:
    for pg in p_g:
      z_p_in_p_g = pg.find(".//z_p_in_p-g") ;
      if z_p_in_p_g is not None:
        wordtype = z_p_in_p_g.text.strip()
      else: return thongtin
      thongtin[wordtype] =  []
      n_g = pg.findall(".//n-g")
      if not n_g :
        if pg.find(".//d") : d_ud = pg.find(".//d");
        elif pg.find(".//ud") : d_ud = pg.find(".//ud");
        elif pg.find(".//xr") : d_ud = pg.find(".//xr")
        if d_ud: em, vm,ex = meaningex(root, d_ud) ; thongtin[wordtype].append({"em":em, "vm":vm, "ex": ex})
      else:
        for ng in n_g:
          if findfather(root, ng).tag not in ['pv-g','id-g']:
            zn=ng.find(".//zn")
            if ng.find(".//d") : d_ud = ng.find(".//d");
            elif ng.find(".//ud") : d_ud = ng.find(".//ud");
            elif ng.find(".//xr") : d_ud = ng.find(".//xr")
            if d_ud: em, vm, ex = meaningex(root, d_ud); thongtin[wordtype].append({"em":em, "vm":vm, "ex":ex});
          d_ud=None

  #print(f"Thông tin của {word} là = ", thongtin)
  return thongtin  # thongtin là 1 đối tượng kiểu dictionary key là phiên âm pronunciation và từ loại wordtype:noun, verb, adj., adv., values của các keys này là nghĩa anh em, nghĩa việt, VD ex


# input: 1 từ điển của 1 từ --> output: là từ điển chỉ giữ lại các wordtype-[{"em":đn tiếng anh},{"em: đn tiếng anh"}, ...]
def em_list_dic(tudien1tu):
  kq={"noun":[], "verb":[], "adj.":[], "adv.":[]}
  for word_type, values  in tudien1tu.items():
    mang_em=[]
    if word_type in ["pronunciation","synset_id"]: continue
    for i in range(len(values)):
      if values[i]["em"] :
        mang_em.append({"em":values[i]["em"]})
    kq[word_type] = mang_em

  # trước khi trả về giá trị kq, xóa đi các từ loại mà không có giá trị
  keys_to_remove = []
  for key, value in kq.items():
    if not value:    keys_to_remove.append(key)
  for key in keys_to_remove:  del kq[key]
  return kq


Phần 3) Xây dựng từ điển EE từ WordNet   03-09-2024

In [9]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
import json
import os

# Hàm để tạo từ điển Anh-Anh từ WordNet: nhập vào 1 từ, trả về các cặp key-value là synset_id - {thông tin loại từ, định nghĩa, đồng nghĩa, trái nghĩa, cha, con, VD}
def dictionary_1word_wordnet(word):
  wordresult = {"noun": [], "verb": [], "adj.": [], "adv.": []}
  synsets = wordnet.synsets(word);
  for synset in synsets:                                       # Duyệt qua tất cả synsets và thu thập thông tin
    synset_id = str(synset.name());
    wordtype = synset.pos()
    if wordtype == 'n' or wordtype == "s": wordtype="noun" ;
    elif wordtype == 'v': wordtype="verb" ;
    elif wordtype == 'a': wordtype="adj." ;
    elif wordtype == 'r': wordtype="adv." ;

    em = synset.definition()
    synonyms = [lemma.name() for lemma in synset.lemmas()]
    antonyms_kieu_lemma= synset.lemmas()[0].antonyms()
    antonyms = [antonym.name() for antonym in antonyms_kieu_lemma]
    parents = [str(parent.name()) for parent in synset.hypernyms()]
    childrens = [str(child.name()) for child in synset.hyponyms()]
    ex = synset.examples()
    wordresult[wordtype].append({"synset_id": synset_id, "wordtype": wordtype,"em": em, 'synonyms': synonyms,"antonyms": antonyms,'parents':parents, 'children': childrens,"ex": ex} ) # Thêm thông tin vào từ điển
  keys_to_remove = []
  for key, value in wordresult.items():
    if not value:    keys_to_remove.append(key)
  for key in keys_to_remove:  del wordresult[key]
  return wordresult




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Phần 4) Tạo từ điển

In [ ]:
# Viết hàm sắp xếp thứ tự mảng các chuỗi theo A,a, B, b...
def custom_sort(s):
    return (s[0].lower(), not s[0].isupper())


# Sắp xếp 1 mảng các từ tiếng anh, trả về theo thứ tự alphabe như trong mảng TT bên dưới, TT có ký tự khoảng trắng nằm cuối cùng
def compare_words(word1, word2,TT = '\'_-./ 0123456789AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZz'):
    min_length = min(len(word1), len(word2))
    for i in range(min_length):
        index1 = TT.index(word1[i])
        index2 = TT.index(word2[i])
        if index1 < index2:
            return -1
        elif index1 > index2:
            return 1

    # Nếu đến đây, hai từ có cùng phần đầu, so sánh theo độ dài
    if len(word1) < len(word2):
        return -1
    elif len(word1) > len(word2):
        return 1
    else:
        return 0


def bubble_sort(words): # nhập vào mảng các từ, trả về mảng đã được sắp xếp
    n = len(words)
    for i in range(n - 1):
        for j in range(n - i - 1):
            if compare_words(words[j], words[j + 1]) > 0:
                words[j], words[j + 1] = words[j + 1], words[j]
    return words


def quick_sort(words, compare_func):
    def partition(low, high):
        pivot = words[high]
        i = low - 1
        for j in range(low, high):
            if compare_func(words[j], pivot) <= 0:
                i += 1
                words[i], words[j] = words[j], words[i]
        words[i + 1], words[high] = words[high], words[i + 1]
        return i + 1

    def quick_sort_recursive(low, high):
        if low < high:
            pi = partition(low, high)
            quick_sort_recursive(low, pi - 1)
            quick_sort_recursive(pi + 1, high)

    quick_sort_recursive(0, len(words) - 1)
    return words


def count_wordnet(): #TRẢ về có bao nhiêu phần tử bắt đầu bằng ký tự kytu trong ds tat cac cac tu cua wordnet, Trả về vị trí bắt đầu và vị trí kết thúc của các phần tử bắt đầu cùa từng vần A, B...
  all_words_set = set()
  for synset in wordnet.all_synsets():
    for lemma in synset.lemmas():
      all_words_set.add(lemma.name())
  all_words_list = quick_sort(list(all_words_set), compare_words ) # sắp xếp A, a, B, b, ...

  chucaihoa=["A","B","C","D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
  chucaithuong= [ "a", "b" , "c" , "d", "e", "f", "g", "h", "i", "J", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
  ketquadem=[0]*27            # Tạo mảng 27 phần tử có giá trị ban đầu là 0
  vitribd=[0]*27
  vitrikt=[0]*27

   # Đếm số lượng phần tử bắt đầu bằng chữ 'A', 'B', ...
  for i in range(len(chucaihoa)):
    kytu=chucaihoa[i]
    for j in all_words_set:
      if j.startswith(kytu) == True :  ketquadem[i]=ketquadem[i]+1

  for i in range(len(chucaithuong)):
    kytu=chucaithuong[i]
    for j in all_words_set:
      if j.startswith(kytu) == True :  ketquadem[i]=ketquadem[i]+1

  ketquadem[i+1] = len(all_words_set) - sum(ketquadem)

  # Tìm phần tử đầu tiên bắt đầu bằng ký tự target_char
  def FindFchar(array, char):
    for item in array:
        if item.startswith(char):
            return item, array.index(item)
    return None

  for k  in range(len(chucaihoa)):
    for f  in range(len(all_words_list)):
      if all_words_list[f].startswith(chucaihoa[k]):
        vitribd[k] = f
        vitrikt[k] = f + ketquadem[k] #- 1
        break

  vitribd[26] = 0
  vitrikt[26] = vitribd[0]-1

  return all_words_list, len(all_words_list), ketquadem, vitribd, vitrikt


def dictionary_wordnet(fr, to):                # nhập vào giá trị từ số --> tới số , trả về từ điển wordnet gồm tất cả các từ
  all_words_set = set()
  for synset in wordnet.all_synsets():
    for lemma in synset.lemmas():
      all_words_set.add(lemma.name())
  all_words_list = quick_sort(list(all_words_set), compare_words)

  # Lấy 1 vài từ trong danh sách,
  words = all_words_list[fr:to]    ;
  EE = {}
  for word in words:
    EE[word] = dictionary_1word_wordnet(word)    # thêm từ điển của 1 từ vào EEV
  return EE

# --------------------------------------- Nhập đường dẫn File Oxford vào -------------------------------------------------------------------------

#Hàm nhập vào 1 file từ điển xml của oxford, trả về từ điển tích hợp wordnet và xml
def intergration_dic(file_paths=["/any.xml"]):
  #file_paths = ["/any.xml"]
  root_xml, word_list_xml = wordlist(file_paths)  # hàm trả về root và danh sách các từ trong file xml
  #print("root_xml = " , root_xml)

  # lấy thông tin của từ điển WORDNET
  word_list_wn, soluongtu, ketquadem, vitribd, vitrikt=count_wordnet() ;

  # Xác định vitribd và vitrikt dựa vào từ đầu tiên của file xml
  chucaihoa=["A","B","C","D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
  vitri=chucaihoa.index(word_list_xml[0][0])
  EE_any_wn=dictionary_wordnet(vitribd[vitri], vitrikt[vitri])

  # Tìm các từ xml có trong wordnet
  result = [word for word in word_list_xml if word in word_list_wn]

  word_list_wn = result   #############################################################################

  # xét từng từ của wordnet và tìm thongtin_tudien của từ này trong Oxford XML
  for word_wn in word_list_wn:
    print("word is: ", word_wn)
    thongtin1tu_wn=dictionary_1word_wordnet(word_wn) ;
    em_list_wn = em_list_dic(thongtin1tu_wn)
    if word_wn in word_list_xml:
      thongtin1tu_xml=dictionary_1word_xml(root_xml, word_wn) ;
      em_list_xml = em_list_dic(thongtin1tu_xml)  ;
      EE_any_wn[word_wn]["pronunciation"] = thongtin1tu_xml["pronunciation"]

  # so sánh độ tương đồng 2 mảng định nghĩa tiếng anh
    for loaitu1, em_dn1 in em_list_wn.items():
      wn = em_list_wn[loaitu1]       ;
      for loaitu2, em_dn2 in em_list_xml.items():
        xml= em_list_xml[loaitu2]
        if loaitu1 == loaitu2:    # nếu cả 2 cùng từ loại noun, hoặc verb, ...
          mangdotd=find_high_similarity_pairs(wn, xml, threshold=0.83)
          for dotd in mangdotd:
              EE_any_wn[word_wn][loaitu1][dotd["Index_List1"]]["em_ox"] = thongtin1tu_xml[loaitu2][dotd["Index_List2"]]["em"]
              EE_any_wn[word_wn][loaitu1][dotd["Index_List1"]]["vm_ox"] = thongtin1tu_xml[loaitu2][dotd["Index_List2"]]["vm"]
              EE_any_wn[word_wn][loaitu1][dotd["Index_List1"]]["ex_ox"] = thongtin1tu_xml[loaitu2][dotd["Index_List2"]]["ex"]
          break

  # Tạo 1 file sẽ lưu kết quả cuối cùng của từ điển tích hợp wordnet và oxford
  with open('/finaldic_any.json', 'w') as file:
    file.write('{}')

 # Lưu kết quả vào file finadic_any.json
  with open("/finaldic_any.json", 'w', encoding='utf-8') as file:
    json.dump(EE_any_wn, file, ensure_ascii=False, indent=4)

  duong_dan= "/finaldic_any.json"
  #duong_dan=os.path.join(os.getcwd(), "finaldic_any.json")
  return duong_dan




Phần 5) Nhập thông tin

In [ ]:
# ---------------- người dùng Chỉ cần nhập đường dẫn vào thế chỗ "/a.xml", kết quả trả về là đường dẫn lưu file "/finaldic_any.json" -----------------------
duongdankq=intergration_dic(["/any.xml"])
print(" ***** Đường dẫn lưu kết quả là = ",duongdankq)


Phần 6) Tạo từ điển dạng csv và excel

In [ ]:
import json
import pandas as pd

# Đọc file JSON
with open('finaldic_any.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Chuẩn bị một DataFrame rỗng để lưu trữ tất cả dữ liệu
all_data = []

# Xử lý từng mục trong dữ liệu JSON
for word, details in data.items():
    # Lấy phần phát âm
    pronunciation = details.get("pronunciation", [])
    pronunciation_value = pronunciation[0] if pronunciation else ""

    # Lặp qua danh sách danh từ
    for noun in details.get("noun", []):
        noun['word'] = word  # Thêm từ vào từng mục
        noun['pronunciation'] = pronunciation_value  # Thêm phát âm

        # Thêm các trường bổ sung nếu có
        noun['em_ox'] = noun.get('em_ox', "")
        noun['vm_ox'] = noun.get('vm_ox', "")
        noun['ex_ox'] = noun.get('ex_ox', [])

        all_data.append(noun)

# Chuyển đổi dữ liệu thành DataFrame
df = pd.DataFrame(all_data)

# Sắp xếp lại cột: 'word', 'pronunciation', 'synset_id', 'wordtype', v.v.
df = df[['word', 'pronunciation', 'synset_id', 'wordtype', 'em', 'synonyms', 'antonyms', 'parents', 'children', 'ex', 'em_ox', 'vm_ox', 'ex_ox']]

# Lưu thành file CSV
df.to_csv('finaldic_any.csv', index=False)

# Lưu thành file Excel
df.to_excel('finaldic_any.xlsx', index=False, engine='openpyxl')

print("Files have been created: data.csv and data.xlsx")